In [53]:
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
import time



from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer


import warnings 
warnings.filterwarnings('ignore')

In [54]:
df1 = pd.read_csv("train.csv")
df1.head()

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,1,126,1931,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,86,93.0,86.0,NaN,NaN,0.01,0.00,0,0,0
1,2,112,1544,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,95,95.0,95.0,333.0,3.7,NaN,NaN,0,0,0
2,3,153,1517,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,162,174.0,162.0,160.0,4.2,0.38,0.15,0,0,0
3,4,109,1811,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,140,163.0,140.0,258.0,3.8,0.12,0.06,0,0,0
4,5,287,1845,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,119,119.0,119.0,110.0,3.9,0.15,0.07,0,0,0


In [55]:
df2 = pd.read_csv('test.csv')
df2

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis
0,50001,267,1658,Caucasian,F,Accident & Emergency,admit,CCU-CTICU,Cardiovascular,Cardiovascular,...,167,121,167.0,121.0,117.0,4.4,0.02,0.01,0,0
1,50002,161,1487,Other/Unknown,M,Accident & Emergency,admit,Neuro ICU,Neurological,Neurologic,...,159,94,159.0,94.0,136.0,3.5,0.85,0.80,0,0
2,50003,287,1845,Caucasian,M,Floor,transfer,CSICU,Cardiovascular,Cardiovascular,...,126,108,126.0,108.0,228.0,4.2,0.15,0.07,0,0
3,50004,170,1373,Other/Unknown,F,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,142,127,142.0,127.0,443.0,3.2,0.09,0.04,0,0
4,50005,276,1747,African American,F,Accident & Emergency,admit,CCU-CTICU,Respiratory,Respiratory,...,110,110,110.0,110.0,219.0,3.6,0.13,0.07,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,79996,276,1723,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,146,146,146.0,146.0,112.0,4.0,0.05,0.02,0,0
29996,79997,207,1343,Hispanic,M,Floor,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,151,151,122.0,122.0,139.0,4.1,0.07,0.02,0,0
29997,79998,251,1506,Caucasian,F,Accident & Emergency,admit,Med-Surg ICU,Genitourinary,Renal/Genitourinary,...,131,129,131.0,129.0,155.0,4.9,0.06,0.02,0,0
29998,79999,209,1129,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Gastrointestinal,Gastrointestinal,...,121,98,121.0,98.0,194.0,3.4,0.02,0.01,1,0


In [ ]:
#Imputer for categorical columns with most frequent value
from sklearn.impute import SimpleImputer
Cat_columns =['ethnicity', 'gender','icu_admit_source','icu_stay_type','icu_type','apache_3j_bodysystem','apache_2_bodysystem', 'elective_surgery','apache_post_operative','gcs_eyes_apache','gcs_unable_apache','gcs_verbal_apache','intubated_apache','ventilated_apache','immunosuppression','solid_tumor_with_metastasis']
imputer = SimpleImputer(strategy='most_frequent')
df1[Cat_columns] = imputer.fit_transform(df1[Cat_columns])


#Imputer for categorical columns with most frequent value
from sklearn.impute import SimpleImputer
Cat_columns =['ethnicity', 'gender','icu_admit_source','icu_stay_type','icu_type','apache_3j_bodysystem','apache_2_bodysystem', 'elective_surgery','apache_post_operative','gcs_eyes_apache','gcs_unable_apache','gcs_verbal_apache','intubated_apache','ventilated_apache','immunosuppression','solid_tumor_with_metastasis']
imputer = SimpleImputer(strategy='most_frequent')
df2[Cat_columns] = imputer.fit_transform(df2[Cat_columns])

from sklearn.impute import SimpleImputer
Cat_columns =['RecordID', 'hospital_id','icu_id','heart_rate_apache','resprate_apache','d1_diasbp_min','d1_diasbp_noninvasive_min','d1_sysbp_min','h1_diasbp_min','h1_diasbp_noninvasive_min','h1_heartrate_max','h1_heartrate_min','h1_mbp_min','h1_mbp_noninvasive_max','h1_mbp_noninvasive_min','h1_sysbp_max','h1_sysbp_min','h1_sysbp_noninvasive_min']
imputer = SimpleImputer(strategy='mean')
df1[Cat_columns] = imputer.fit_transform(df1[Cat_columns])

from sklearn.impute import SimpleImputer
Cat_columns =['RecordID', 'hospital_id','icu_id','heart_rate_apache','resprate_apache','d1_diasbp_min','d1_diasbp_noninvasive_min','d1_sysbp_min','h1_diasbp_min','h1_diasbp_noninvasive_min','h1_heartrate_max','h1_heartrate_min','h1_mbp_min','h1_mbp_noninvasive_max','h1_mbp_noninvasive_min','h1_sysbp_max','h1_sysbp_min','h1_sysbp_noninvasive_min']
imputer = SimpleImputer(strategy='mean')
df2[Cat_columns] = imputer.fit_transform(df2[Cat_columns])
sb

from sklearn.impute import SimpleImputer
Cat_columns =['age', 'pre_icu_los_days','apache_2_diagnosis','apache_3j_diagnosis','gcs_motor_apache','temp_apache','d1_heartrate_max','d1_mbp_min','d1_mbp_noninvasive_min','d1_resprate_max','d1_spo2_min','d1_sysbp_noninvasive_min','d1_temp_min','h1_mbp_max','h1_resprate_max','h1_resprate_min','h1_spo2_max','h1_spo2_min','h1_sysbp_noninvasive_max','d1_glucose_max','d1_potassium_max','apache_4a_hospital_death_prob','apache_4a_icu_death_prob']
imputer = SimpleImputer(strategy='median')
df1[Cat_columns] = imputer.fit_transform(df1[Cat_columns])

from sklearn.impute import SimpleImputer
Cat_columns =['age', 'pre_icu_los_days','apache_2_diagnosis','apache_3j_diagnosis','gcs_motor_apache','temp_apache','d1_heartrate_max','d1_mbp_min','d1_mbp_noninvasive_min','d1_resprate_max','d1_spo2_min','d1_sysbp_noninvasive_min','d1_temp_min','h1_mbp_max','h1_resprate_max','h1_resprate_min','h1_spo2_max','h1_spo2_min','h1_sysbp_noninvasive_max','d1_glucose_max','d1_potassium_max','apache_4a_hospital_death_prob','apache_4a_icu_death_prob']
imputer = SimpleImputer(strategy='median')
df2[Cat_columns] = imputer.fit_transform(df2[Cat_columns])



In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoders ={}

# Apply label encoding to the categorical columns in your DataFrame
categorical_columns = ['ethnicity', 'gender','icu_admit_source','icu_stay_type','icu_type','apache_3j_bodysystem','apache_2_bodysystem', 'elective_surgery','apache_post_operative','gcs_eyes_apache','gcs_unable_apache','gcs_verbal_apache','intubated_apache','ventilated_apache','immunosuppression','solid_tumor_with_metastasis']
  # Replace with your column names
for column in categorical_columns:
    label_encoder = LabelEncoder()
    df1[column] = label_encoder.fit_transform(df1[column].values)
    label_encoders[column] = label_encoder

# Now, the categorical columns in both DataFrames are label-encoded


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoders_test ={}

# Apply label encoding to the categorical columns in your DataFrame
categorical_columns = ['ethnicity', 'gender','icu_admit_source','icu_stay_type','icu_type','apache_3j_bodysystem','apache_2_bodysystem', 'elective_surgery','apache_post_operative','gcs_eyes_apache','gcs_unable_apache','gcs_verbal_apache','intubated_apache','ventilated_apache','immunosuppression','solid_tumor_with_metastasis']
  # Replace with your column names
for column in categorical_columns:
    label_encoder = LabelEncoder()
    df2[column] = label_encoder.fit_transform(df2[column].values)
    label_encoders_test[column] = label_encoder


# Now, the categorical columns in both DataFrames are label-encoded


In [ ]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)






In [ ]:
X = df_onehot.drop(columns=['hospital_death','RecordID','hospital_id','icu_id','ethnicity_African American','ethnicity_Asian','ethnicity_Caucasian','ethnicity_Hispanic','ethnicity_Native American','ethnicity_Other/Unknown','gender_F','gender_M','icu_admit_source_Accident & Emergency','icu_admit_source_Other ICU','icu_stay_type_readmit','icu_type_CTICU','icu_type_Med-Surg ICU','icu_type_Neuro ICU','apache_3j_bodysystem_Cardiovascular','apache_3j_bodysystem_Hematological','apache_2_bodysystem_Haematologic','apache_2_bodysystem_Undefined Diagnoses','h1_spo2_max','immunosuppression_0','immunosuppression_1','icu_admit_source_Other Hospital','icu_stay_type_admit','icu_stay_type_transfer','icu_type_CCU-CTICU','icu_type_CSICU','icu_type_Cardiac ICU','icu_type_MICU','icu_type_SICU','apache_3j_bodysystem_Gastrointestinal','apache_3j_bodysystem_Genitourinary','apache_3j_bodysystem_Gynecological','apache_3j_bodysystem_Musculoskeletal/Skin','apache_3j_bodysystem_Neurological','apache_3j_bodysystem_Respiratory','apache_3j_bodysystem_Trauma','apache_2_bodysystem_Neurologic','apache_2_bodysystem_Renal/Genitourinary','apache_2_bodysystem_Respiratory','apache_2_bodysystem_Trauma','apache_2_bodysystem_Undefined diagnoses','pre_icu_los_days','apache_2_diagnosis','apache_3j_diagnosis','resprate_apache','h1_heartrate_min','h1_mbp_max','h1_mbp_noninvasive_max','h1_sysbp_max','h1_sysbp_noninvasive_max','d1_glucose_max','solid_tumor_with_metastasis_0','solid_tumor_with_metastasis_1','icu_admit_source_Floor','icu_admit_source_Operating Room / Recovery','icu_admit_source_Other Hospital','apache_3j_bodysystem_Metabolic','apache_3j_bodysystem_Sepsis','apache_2_bodysystem_Cardiovascular','apache_2_bodysystem_Metabolic'])
y = df_onehot[['hospital_death']]

df2_encoded = test_onehot.drop(columns=['RecordID','hospital_id','icu_id','ethnicity_African American','ethnicity_Asian','ethnicity_Caucasian','ethnicity_Hispanic','ethnicity_Native American','ethnicity_Other/Unknown','gender_F','gender_M','icu_admit_source_Accident & Emergency','icu_admit_source_Other ICU','icu_stay_type_readmit','icu_type_CTICU','icu_type_Med-Surg ICU','icu_type_Neuro ICU','apache_3j_bodysystem_Cardiovascular','apache_3j_bodysystem_Hematological','apache_2_bodysystem_Haematologic','apache_2_bodysystem_Undefined Diagnoses','h1_spo2_max','immunosuppression_0','immunosuppression_1','icu_admit_source_Other Hospital','icu_stay_type_admit','icu_stay_type_transfer','icu_type_CCU-CTICU','icu_type_CSICU','icu_type_Cardiac ICU','icu_type_MICU','icu_type_SICU','apache_3j_bodysystem_Gastrointestinal','apache_3j_bodysystem_Genitourinary','apache_3j_bodysystem_Gynecological','apache_3j_bodysystem_Musculoskeletal/Skin','apache_3j_bodysystem_Neurological','apache_3j_bodysystem_Respiratory','apache_3j_bodysystem_Trauma','apache_2_bodysystem_Neurologic','apache_2_bodysystem_Renal/Genitourinary','apache_2_bodysystem_Respiratory','apache_2_bodysystem_Trauma','apache_2_bodysystem_Undefined diagnoses','pre_icu_los_days','apache_2_diagnosis','apache_3j_diagnosis','resprate_apache','h1_heartrate_min','h1_mbp_max','h1_mbp_noninvasive_max','h1_sysbp_max','h1_sysbp_noninvasive_max','d1_glucose_max','solid_tumor_with_metastasis_0','solid_tumor_with_metastasis_1','icu_admit_source_Floor','icu_admit_source_Operating Room / Recovery','icu_admit_source_Other Hospital','apache_3j_bodysystem_Metabolic','apache_3j_bodysystem_Sepsis','apache_2_bodysystem_Cardiovascular','apache_2_bodysystem_Metabolic'])


In [ ]:
X = df1.drop(columns=['hospital_death'])  # Exclude the target column
y = df1[['hospital_death']]

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs_forward = SequentialFeatureSelector(DecisionTreeClassifier(max_depth= 7, min_samples_split= 5, min_samples_leaf=300),
                                        n_features_to_select=5,
                                        direction='forward',
                                        scoring='roc_auc').fit(X,y)


sfs_forward.fit(X,y)
X = sfs_forward.transform(X)
df2_trans = sfs_forward.transform(df2)

In [ ]:
knn_imputer = KNNImputer(n_neighbors=20)
X = knn_imputer.fit_transform(X)
df2_impu_knn = knn_imputer.fit_transform(df2_trans)

knn_imputer = KNNImputer(n_neighbors=20)
X = knn_imputer.fit_transform(X)
df1_impu_knn = knn_imputer.fit_transform(df_onehot)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
df2_KNN_scaled = scaler.fit_transform(df2_impu_knn)

In [ ]:
from sklearn.preprocessing import RobustScaler
import pandas as pd

# Sample data (replace this with your dataset)




# Initialize the RobustScaler
scaler = RobustScaler()

# Fit and transform the data
scaled_data = scaler.fit_transform(df2_encoded)

# Create a new DataFrame with the scaled data
scaled_df = pd.DataFrame(scaled_data, columns=df2_encoded.columns)

# Display the scaled DataFrame



In [ ]:
dt_5_5 = DecisionTreeClassifier(max_depth=7, min_samples_split= 5 , min_samples_leaf=300)  
dt_5_5.fit(X,y)
#fit_model(dt_5_2, "Depth 5 Split 2") 
md_probs = dt_5_5.predict_proba(scaled_df)







In [ ]:
md_probs

In [ ]:
md_probs = md_probs[:,1]
additional_data = pd.read_csv("test.csv")
predicted_df = pd.DataFrame(md_probs, columns=['hospital_death'])
predicted_df.to_csv('prediction30.csv', index=False)

md_probs